In [141]:
import pandas as pd
import requests
import bs4
import re

In [167]:
def getWikipediaList(url, table_index):
    
    page = requests.get(url)
    soup = bs4.BeautifulSoup(page.content, 'html.parser')
    tables = soup.find_all('table')
    table = tables[table_index]
    table_headers = []
    for tx in table.find_all('th'):
        table_headers.append(re.sub('\[\d+\]','',tx.text.replace('\n', '')))
    result = pd.DataFrame([[td.text for td in row.find_all('td')] for row in table.tbody.find_all('tr')])
    print(table_headers)
    result.columns = table_headers
    result.replace(r'\s', '', regex = True, inplace = True)
    result.drop([0], inplace= True)
    return result

In [168]:
# UNFCC
# EU
# NATO
# OECD
# G7
# UN
# OSCE
# BIS
# COE
# ILO
# INTERPOL
uncfcc_countries = getWikipediaList(url = 'https://en.wikipedia.org/wiki/List_of_parties_to_the_United_Nations_Framework_Convention_on_Climate_Change', table_index=0)
uncfcc_countries

['State', 'Classification', 'Signature', 'Ratification', 'Notes']


,State,Classification,Signature,Ratification,Notes
1,Afghanistan,,"June12,1992","September19,2002",
2,Albania,,,"October3,1994",
3,Algeria,,"June13,1992","June9,1993",
4,Andorra,,,"March2,2011",
5,Angola,,"June14,1992","May17,2000",
6,AntiguaandBarbuda,,"June4,1992","February2,1993",
7,Argentina,,"June12,1992","March11,1994",
8,Armenia,,"June13,1992","May14,1993",
9,Australia,"AnnexI,II","June4,1992","December30,1992",
10,Austria,"AnnexI,II","June8,1992","February28,1994",


In [169]:
eu_countries = getWikipediaList('https://en.wikipedia.org/wiki/Member_state_of_the_European_Union', table_index=1)
eu_countries

['Name', 'Accession', 'Population', 'Area (km²)', 'GDP(US$M)', 'GDP per cap.(PPP)', 'Currency', 'Gini', 'HDI', 'MEPs', 'Languages', 'ISO 3166-1 alpha-2', 'ISO 3166-1 alpha-3', ' Austria', ' Belgium', ' Bulgaria', ' Croatia', ' Cyprus[a]', ' Czech Republic', ' Denmark[d]', ' Estonia', ' Finland[e]', ' France[f]', ' Germany', ' Greece', ' Hungary', ' Ireland', ' Italy', ' Latvia', ' Lithuania', ' Luxembourg', ' Malta', ' Netherlands[i]', ' Poland', ' Portugal[k]', ' Romania', ' Slovakia', ' Slovenia', ' Spain[l]', ' Sweden']


ValueError: Length mismatch: Expected axis has 13 elements, new values have 40 elements

,State,Classification,Signature,Ratification,Notes
1,Afghanistan,,"June12,1992","September19,2002",
2,Albania,,,"October3,1994",
3,Algeria,,"June13,1992","June9,1993",
4,Andorra,,,"March2,2011",
5,Angola,,"June14,1992","May17,2000",
6,AntiguaandBarbuda,,"June4,1992","February2,1993",
7,Argentina,,"June12,1992","March11,1994",
8,Armenia,,"June13,1992","May14,1993",
9,Australia,"AnnexI,II","June4,1992","December30,1992",
10,Austria,"AnnexI,II","June8,1992","February28,1994",


In [ ]:
url = 'https://en.wikipedia.org/wiki/Member_state_of_the_European_Union'

In [76]:
coordinates

,country_code,latitude,longitude,country,usa_state_code,usa_state_latitude,usa_state_longitude,usa_state
0,AD,42.546245,1.601554,Andorra,AK,63.588753,-154.493062,Alaska
1,AE,23.424076,53.847818,United Arab Emirates,AL,32.318231,-86.902298,Alabama
2,AF,33.939110,67.709953,Afghanistan,AR,35.201050,-91.831833,Arkansas
3,AG,17.060816,-61.796428,Antigua and Barbuda,AZ,34.048928,-111.093731,Arizona
4,AI,18.220554,-63.068615,Anguilla,CA,36.778261,-119.417932,California
...,...,...,...,...,...,...,...,...
240,YE,15.552727,48.516388,Yemen,NaN,NaN,NaN,NaN
241,YT,-12.827500,45.166244,Mayotte,NaN,NaN,NaN,NaN
242,ZA,-30.559482,22.937506,South Africa,NaN,NaN,NaN,NaN
243,ZM,-13.133897,27.849332,Zambia,NaN,NaN,NaN,NaN


In [123]:
coordinates = pd.read_csv('assets/world_country_and_usa_states_latitude_and_longitude_values.csv')
coordinates = coordinates[['country_code', 'latitude', 'longitude', 'country']]

In [136]:
coordinates.set_index('country')

,country_code,latitude,longitude
country,,,
Andorra,AD,42.546245,1.601554
United Arab Emirates,AE,23.424076,53.847818
Afghanistan,AF,33.939110,67.709953
Antigua and Barbuda,AG,17.060816,-61.796428
Anguilla,AI,18.220554,-63.068615
Albania,AL,41.153332,20.168331
Armenia,AM,40.069099,45.038189
Netherlands Antilles,AN,12.226079,-69.060087
Angola,AO,-11.202692,17.873887


In [137]:
coordinates.merge(uncfcc_countries, left_on='country', right_on='State')

,country_code,latitude,longitude,country,State,Classification,Signature,Ratification,Notes
0,AD,42.546245,1.601554,Andorra,Andorra,,,"March2,2011",
1,AF,33.939110,67.709953,Afghanistan,Afghanistan,,"June12,1992","September19,2002",
2,AL,41.153332,20.168331,Albania,Albania,,,"October3,1994",
3,AM,40.069099,45.038189,Armenia,Armenia,,"June13,1992","May14,1993",
4,AO,-11.202692,17.873887,Angola,Angola,,"June14,1992","May17,2000",
5,AR,-38.416097,-63.616672,Argentina,Argentina,,"June12,1992","March11,1994",
6,AT,47.516231,14.550072,Austria,Austria,"AnnexI,II","June8,1992","February28,1994",
7,AU,-25.274398,133.775136,Australia,Australia,"AnnexI,II","June4,1992","December30,1992",
8,AZ,40.143105,47.576927,Azerbaijan,Azerbaijan,,"June12,1992","May16,1995",
9,BB,13.193887,-59.543198,Barbados,Barbados,,"June12,1992","March23,1994",


In [128]:
countries = pd.read_csv('assets/countries_of_the_world.csv') #https://www.kaggle.com/fernandol/countries-of-the-world 1970-2017
countries.columns = countries.columns.str.lower()


In [131]:
countries[countries['country'].str.contains(' and |&')]

,country,region,population,area (sq. mi.),pop. density (per sq. mi.),coastline (coast/area ratio),net migration,infant mortality (per 1000 births),gdp ($ per capita),literacy (%),phones (per 1000),arable (%),crops (%),other (%),climate,birthrate,deathrate,agriculture,industry,service
7,Antigua & Barbuda,LATIN AMER. & CARIB,69108,443,"156,0","34,54","-6,15","19,46",11000.0,"89,0","549,9","18,18","4,55","77,27",2,"16,93","5,37","0,038","0,22","0,743"
25,Bosnia & Herzegovina,EASTERN EUROPE,4498976,51129,"88,0","0,04","0,31","21,05",6100.0,NaN,"215,4","13,6","2,96","83,44",4,"8,77","8,27","0,142","0,308","0,55"
172,Saint Kitts & Nevis,LATIN AMER. & CARIB,39129,261,"149,9","51,72","-7,11","14,49",8800.0,"97,0","638,9","19,44","2,78","77,78",2,"18,02","8,33","0,035","0,258","0,707"
174,St Pierre & Miquelon,NORTHERN AMERICA,7026,242,"29,0","49,59","-4,86","7,54",6900.0,"99,0","683,2","13,04",0,"86,96",NaN,"13,52","6,83",NaN,NaN,NaN
175,Saint Vincent and the Grenadines,LATIN AMER. & CARIB,117848,389,"303,0","21,59","-7,64","14,78",2900.0,"96,0","190,9","17,95","17,95","64,1",2,"16,18","5,98","0,1","0,26","0,64"
178,Sao Tome & Principe,SUB-SAHARAN AFRICA,193413,1001,"193,2","20,88","-2,72","43,11",1200.0,"79,3","36,2","6,25","48,96","44,79",2,"40,25","6,47","0,167","0,148","0,684"
204,Trinidad & Tobago,LATIN AMER. & CARIB,1065842,5128,"207,9","7,06","-10,83","24,31",9500.0,"98,6","303,5","14,62","9,16","76,22",2,"12,9","10,57","0,007","0,57","0,423"
208,Turks & Caicos Is,LATIN AMER. & CARIB,21152,430,"49,2","90,47","11,68","15,67",9600.0,"98,0","269,5","2,33",0,"97,67",2,"21,84","4,21",NaN,NaN,NaN
221,Wallis and Futuna,OCEANIA,16025,274,"58,5","47,08",NaN,NaN,3700.0,"50,0","118,6",5,25,70,2,NaN,NaN,NaN,NaN,NaN


In [ ]:
countries = countries.sort_values(by=['country'])[0:10]
coordinates = coordinates.sort_values(by=['country'])[0:10]
countries

In [122]:
coordinates[coordinates['country'].str.contains("an")]

,country_code,latitude,longitude,country
2,AF,33.939110,67.709953,Afghanistan
5,AL,41.153332,20.168331,Albania
11,AS,-14.270972,-170.132217,American Samoa
3,AG,17.060816,-61.796428,Antigua and Barbuda


In [121]:
coordinates

,country_code,latitude,longitude,country
2,AF,33.939110,67.709953,Afghanistan
5,AL,41.153332,20.168331,Albania
58,DZ,28.033886,1.659626,Algeria
11,AS,-14.270972,-170.132217,American Samoa
0,AD,42.546245,1.601554,Andorra
8,AO,-11.202692,17.873887,Angola
4,AI,18.220554,-63.068615,Anguilla
9,AQ,-75.250973,-0.071389,Antarctica
3,AG,17.060816,-61.796428,Antigua and Barbuda
10,AR,-38.416097,-63.616672,Argentina


In [93]:
pd.set_option('display.max_rows', 500)
countries.merge(coordinates, left_on='country', right_on='country', how='inner')

,country,region,population,area (sq. mi.),pop. density (per sq. mi.),coastline (coast/area ratio),net migration,infant mortality (per 1000 births),gdp ($ per capita),literacy (%),...,other (%),climate,birthrate,deathrate,agriculture,industry,service,country_code,latitude,longitude


In [73]:
countries

,Country,Region,Population,Area (sq. mi.),Pop. Density (per sq. mi.),Coastline (coast/area ratio),Net migration,Infant mortality (per 1000 births),GDP ($ per capita),Literacy (%),Phones (per 1000),Arable (%),Crops (%),Other (%),Climate,Birthrate,Deathrate,Agriculture,Industry,Service
0,Afghanistan,ASIA (EX. NEAR EAST),31056997,647500,"48,0","0,00","23,06","163,07",700.0,"36,0","3,2","12,13","0,22","87,65",1,"46,6","20,34","0,38","0,24","0,38"
1,Albania,EASTERN EUROPE,3581655,28748,"124,6","1,26","-4,93","21,52",4500.0,"86,5","71,2","21,09","4,42","74,49",3,"15,11","5,22","0,232","0,188","0,579"
2,Algeria,NORTHERN AFRICA,32930091,2381740,"13,8","0,04","-0,39",31,6000.0,"70,0","78,1","3,22","0,25","96,53",1,"17,14","4,61","0,101","0,6","0,298"
3,American Samoa,OCEANIA,57794,199,"290,4","58,29","-20,71","9,27",8000.0,"97,0","259,5",10,15,75,2,"22,46","3,27",NaN,NaN,NaN
4,Andorra,WESTERN EUROPE,71201,468,"152,1","0,00","6,6","4,05",19000.0,"100,0","497,2","2,22",0,"97,78",3,"8,71","6,25",NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
222,West Bank,NEAR EAST,2460492,5860,"419,9","0,00","2,98","19,62",800.0,NaN,"145,2","16,9","18,97","64,13",3,"31,67","3,92","0,09","0,28","0,63"
223,Western Sahara,NORTHERN AFRICA,273008,266000,"1,0","0,42",NaN,NaN,NaN,NaN,NaN,"0,02",0,"99,98",1,NaN,NaN,NaN,NaN,"0,4"
224,Yemen,NEAR EAST,21456188,527970,"40,6","0,36",0,"61,5",800.0,"50,2","37,2","2,78","0,24","96,98",1,"42,89","8,3","0,135","0,472","0,393"
225,Zambia,SUB-SAHARAN AFRICA,11502010,752614,"15,3","0,00",0,"88,29",800.0,"80,6","8,2","7,08","0,03","92,9",2,41,"19,93","0,22","0,29","0,489"


In [ ]:
import 